In [1]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))
import datetime
print("This notebook was executed on: \n{}".format(datetime.datetime.now()))

This notebook was run with herschelhelp_internal version: 
1407877 (Mon Feb 4 12:56:29 2019 +0000) [with local modifications]
This notebook was executed on: 
2021-01-14 10:13:41.736847


In [1]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

import os
import time
import gc
import glob

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table, join, vstack, hstack
import numpy as np
from pymoc import MOC

from herschelhelp_internal.masterlist import merge_catalogues, nb_merge_dist_plot, specz_merge
from herschelhelp_internal.utils import coords_to_hpidx, ebv, gen_help_id, inMoc

In [8]:
import gc
gc.collect()

22

## I. Calculate %sources through HELP pipeline

In [2]:
def add_photoz_selection_flag(catalogue, all_bands, ndet=4):
    """
    """
    
    catalogue = catalogue.copy()
    nb_band = np.zeros(len(catalogue))
    r_flux = np.zeros(len(catalogue))
    
    for idx, bands in enumerate(all_bands):
        nb_filter = np.zeros(len(catalogue))
        for filt in bands:
            nb_filter += (1 * ~np.isnan(catalogue[filt]))   
        
            if '_r' in filt:
                r_flux += (1 * ~np.isnan(catalogue[filt])) 


        nb_band += 1 * (nb_filter >= 1)


    bands_det = nb_band >= ndet
    has_r_flux = r_flux >= 1


    catalogue.add_column(
    Column(
        1 * has_r_flux + 2 * bands_det,
        name="flag_photoz_sel_det")
        )

#     catalogue.write("{}{}_{}".format(file_list[][49:81], DATE, file_list[90:]), overwrite=True)

    return catalogue
    

In [11]:
def percentages_help_pipeline_tiles(file_list, all_bands, field):
    """
    """

    n=0
    photoz_r    = 0
    photoz_4b   = 0
    cigale_2opt = 0 
    cigale_2nir = 0 
    
    photoz = 0
    cigale = 0 
    both   = 0
    
    photoz_just_r    = 0
    photoz_just_4b   = 0
    cigale_just_2opt = 0 
    cigale_just_2nir = 0 
    
    total_obj = 0


    for sub_file in range(len(file_list)):
        sub_catalogue = Table.read(file_list[sub_file])

        if 'flag_photoz_sel_det' not in sub_catalogue.colnames:
            sub_catalogue = add_photoz_selection_flag(sub_catalogue, all_bands)

        sources_photoz_4b   = ((sub_catalogue['flag_photoz_sel_det'] == 2) | (sub_catalogue['flag_photoz_sel_det'] == 3))
        sources_photoz_r    = ((sub_catalogue['flag_photoz_sel_det'] == 1) | (sub_catalogue['flag_photoz_sel_det'] == 3))
        sources_cigale_2opt = ((sub_catalogue['flag_optnir_det'] == 1) | (sub_catalogue['flag_optnir_det'] == 3) | 
                               (sub_catalogue['flag_optnir_det'] == 5)| (sub_catalogue['flag_optnir_det'] == 7))
        sources_cigale_2nir = ((sub_catalogue['flag_optnir_det'] == 2) | (sub_catalogue['flag_optnir_det'] == 3) | 
                               (sub_catalogue['flag_optnir_det'] == 6)| (sub_catalogue['flag_optnir_det'] == 7))
        
        sources_photoz = ((sub_catalogue['flag_photoz_sel_det'] == 3) | (~np.isnan(sub_catalogue['zspec'])))
        sources_cigale = ((sub_catalogue['flag_optnir_det'] == 3) | (sub_catalogue['flag_optnir_det'] == 7))
        sources_both   = ((sub_catalogue['flag_photoz_sel_det'] == 3) & ((sub_catalogue['flag_optnir_det'] == 3) | (sub_catalogue['flag_optnir_det'] == 7)))
        
        sources_photoz_just_4b   = (sub_catalogue['flag_photoz_sel_det'] == 2)
        sources_photoz_just_r    = (sub_catalogue['flag_photoz_sel_det'] == 1)
        sources_cigale_just_2opt = ((sub_catalogue['flag_optnir_det'] == 1) | (sub_catalogue['flag_optnir_det'] == 5)) 
        sources_cigale_just_2nir = ((sub_catalogue['flag_optnir_det'] == 2) | (sub_catalogue['flag_optnir_det'] == 6)) 

        photoz_r    += sources_photoz_r.sum() 
        photoz_4b   += sources_photoz_4b.sum() 
        cigale_2opt += sources_cigale_2opt.sum()
        cigale_2nir += sources_cigale_2nir.sum()
        
        photoz += sources_photoz.sum() 
        cigale += sources_cigale.sum()
        both   += sources_both.sum() 
        
        photoz_just_r    += sources_photoz_just_r.sum() 
        photoz_just_4b   += sources_photoz_just_4b.sum() 
        cigale_just_2opt += sources_cigale_just_2opt.sum()
        cigale_just_2nir += sources_cigale_just_2nir.sum()
        
        total_obj += len(sub_catalogue)

        n += 1

    percentage_photoz_r    = photoz_r / total_obj
    percentage_photoz_4b   = photoz_4b / total_obj
    percentage_cigale_2opt = cigale_2opt / total_obj 
    percentage_cigale_2nir = cigale_2nir / total_obj 
    
    percentage_photoz = photoz / total_obj
    percentage_cigale = cigale / total_obj   
    percentage_both   = both / total_obj
    
    percentage_photoz_just_r    = photoz_just_r / total_obj
    percentage_photoz_just_4b   = photoz_just_4b / total_obj
    percentage_cigale_just_2opt = cigale_just_2opt / total_obj
    percentage_cigale_just_2nir = cigale_just_2nir / total_obj

    print("field: ", field)
    print("total objects in masterlist: ", total_obj)
    print("percentage sources photoz_4band sel: {:.3%}".format(percentage_photoz_r))
    print("percentage sources photoz_r sel: {:.3%}".format(percentage_photoz_4b))
    print("percentage sources cigale_2opt sel: {:.3%}".format(percentage_cigale_2opt))
    print("percentage sources cigale_2nir sel: {:.3%}".format(percentage_cigale_2nir))
    print("||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||")
    print("percentage sources photoz sel: {:.3%}".format(percentage_photoz))
    print("percentage sources cigale sel: {:.3%}".format(percentage_cigale))
    print("percentage sources both sel: {:.3%}".format(percentage_both))
    print("||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||")
    print("percentage sources with just photoz_4bands (not r detection): {:.3%}".format(percentage_photoz_just_r))
    print("percentage sources with just photoz_r (not 4bands detection): {:.3%}".format(percentage_photoz_just_4b))
    print("percentage sources with just cigale_2opt (not 2nir detection): {:.3%}".format(percentage_cigale_just_2opt))
    print("percentage sources with just cigale_2nir (not 2opt detection): {:.3%}".format(percentage_cigale_just_2nir))
    
    
    return total_obj, photoz, cigale, both

In [4]:
SUFFIX = '20180307'
file_list = glob.glob('../../dmu1/dmu1_ml_Herschel-Stripe-82/data/tiles/sub_catalogue_herschel-stripe-82_{}_*.fits'.format(SUFFIX))


In [5]:
cat = Table.read(file_list[0])
bands = []
for col in cat.colnames:
    if col.startswith('f_') and not col.startswith('f_ap'):
        bands.append(col)
bands

['f_suprime_g',
 'f_suprime_r',
 'f_suprime_i',
 'f_suprime_z',
 'f_suprime_y',
 'f_suprime_n921',
 'f_suprime_n816',
 'f_vista_y',
 'f_vista_h',
 'f_wircam_j',
 'f_vista_j',
 'f_wircam_ks',
 'f_vista_ks',
 'f_ukidss_y',
 'f_ukidss_j',
 'f_ukidss_h',
 'f_ukidss_k',
 'f_gpc1_g',
 'f_gpc1_r',
 'f_gpc1_i',
 'f_gpc1_z',
 'f_gpc1_y',
 'f_irac_i1',
 'f_irac_i2',
 'f_decam_i',
 'f_decam_y',
 'f_decam_g',
 'f_decam_r',
 'f_decam_z',
 'f_megacam_g',
 'f_megacam_r',
 'f_megacam_i',
 'f_megacam_z',
 'f_megacam_y',
 'f_sdss_u',
 'f_sdss_g',
 'f_sdss_r',
 'f_sdss_i',
 'f_sdss_z']

In [6]:
all_bands = [
 ['f_sdss_u'],
    ['f_suprime_g',
 #'f_gpc1_g',
 'f_decam_g',
 #'f_megacam_g',
 'f_sdss_g'],
 ['f_suprime_r',
 #'f_gpc1_r',
 'f_decam_r',
 #'f_megacam_r',
 'f_sdss_r'],
 ['f_suprime_i',
 #'f_gpc1_i',
 'f_decam_i',
 #'f_megacam_i',
 'f_sdss_i'],
 ['f_suprime_z',
 #'f_gpc1_z',
 'f_decam_z',
 #'f_megacam_z',
 'f_sdss_z'],
# 'f_suprime_n921',
# 'f_suprime_n816',
 ['f_suprime_y',
 'f_vista_y',
 'f_ukidss_y',
 #'f_gpc1_y',
 'f_decam_y'],
 #'f_megacam_y'],
 ['f_vista_h',
 'f_ukidss_h'],
 #['f_wircam_j',
 ['f_ukidss_j',
 'f_vista_j'],
 #['f_wircam_ks',
 ['f_vista_ks',
 'f_ukidss_k'],
 ['f_irac_i1'],
 ['f_irac_i2']
]





In [12]:
total_obj, photoz_hs82, cigale_hs82, both_hs82 = percentages_help_pipeline_tiles(file_list, all_bands, 'Herschel-Stripe-82')

field:  Herschel-Stripe-82
total objects in masterlist:  50196455
percentage sources photoz_4band sel: 54.929%
percentage sources photoz_r sel: 43.475%
percentage sources cigale_2opt sel: 58.565%
percentage sources cigale_2nir sel: 9.569%
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
percentage sources photoz sel: 42.512%
percentage sources cigale sel: 8.121%
percentage sources both sel: 7.099%
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
percentage sources with just photoz_4bands (not r detection): 12.450%
percentage sources with just photoz_r (not 4bands detection): 0.996%
percentage sources with just cigale_2opt (not 2nir detection): 50.444%
percentage sources with just cigale_2nir (not 2opt detection): 1.448%


In [68]:
# Check sources in cigale catalogue
cigale_cat = Table.read('../../dmu28/dmu28_Herschel-Stripe-82/data/herschel_stripe_82_Ldust_prediction.fits', memmap=True)

# Check sources in photo-z catalogue
photoz_cat = Table.read('../../dmu24/dmu24_Herschel-Stripe-82/data/master_catalogue_herschel-stripe-82_20180307_photoz_20180509.fits', memmap=True)

In [69]:
len(cigale_cat) / total_obj * 100

6.891689861365707

In [70]:
len(photoz_cat) / total_obj * 100

43.05846100088144

In [72]:
total_obj * 0.005

250982.275

In [13]:
def percentages_help_pipeline(masterlist, all_bands, field):
    """
    """
    # ref: 2.4_Merging.ipynb
    split_length = 100000 #number of sources to include in every sub catalogue
    num_files = int(np.ceil(len(masterlist)/split_length))
    print('num files: ', num_files)

    n=0
    photoz_r    = 0
    photoz_4b   = 0
    cigale_2opt = 0 
    cigale_2nir = 0 
    
    photoz = 0
    cigale = 0 
    both   = 0
    
    photoz_just_r    = 0
    photoz_just_4b   = 0
    cigale_just_2opt = 0 
    cigale_just_2nir = 0 
    
    total_obj = 0

    for sub_file in range(num_files):
        # ref: 2.4_Merging.ipynb
        sub_catalogue = masterlist[n*split_length:(n+1)*split_length].copy()
        #print(n)

        if 'flag_photoz_sel_det' not in sub_catalogue.colnames:
            sub_catalogue = add_photoz_selection_flag(sub_catalogue, all_bands)

        sources_photoz_4b   = ((sub_catalogue['flag_photoz_sel_det'] == 2) | (sub_catalogue['flag_photoz_sel_det'] == 3))
        sources_photoz_r    = ((sub_catalogue['flag_photoz_sel_det'] == 1) | (sub_catalogue['flag_photoz_sel_det'] == 3))
        sources_cigale_2opt = ((sub_catalogue['flag_optnir_det'] == 1) | (sub_catalogue['flag_optnir_det'] == 3) | 
                               (sub_catalogue['flag_optnir_det'] == 5)| (sub_catalogue['flag_optnir_det'] == 7))
        sources_cigale_2nir = ((sub_catalogue['flag_optnir_det'] == 2) | (sub_catalogue['flag_optnir_det'] == 3) | 
                               (sub_catalogue['flag_optnir_det'] == 6)| (sub_catalogue['flag_optnir_det'] == 7))
        
        sources_photoz = ((sub_catalogue['flag_photoz_sel_det'] == 3) | (~np.isnan(sub_catalogue['zspec'])))
        sources_cigale = ((sub_catalogue['flag_optnir_det'] == 3) | (sub_catalogue['flag_optnir_det'] == 7))
        sources_both   = ((sub_catalogue['flag_photoz_sel_det'] == 3) & ((sub_catalogue['flag_optnir_det'] == 3) | (sub_catalogue['flag_optnir_det'] == 7)))
        
        sources_photoz_just_4b   = (sub_catalogue['flag_photoz_sel_det'] == 2)
        sources_photoz_just_r    = (sub_catalogue['flag_photoz_sel_det'] == 1)
        sources_cigale_just_2opt = ((sub_catalogue['flag_optnir_det'] == 1) | (sub_catalogue['flag_optnir_det'] == 5)) 
        sources_cigale_just_2nir = ((sub_catalogue['flag_optnir_det'] == 2) | (sub_catalogue['flag_optnir_det'] == 6)) 

        photoz_r    += sources_photoz_r.sum() 
        photoz_4b   += sources_photoz_4b.sum() 
        cigale_2opt += sources_cigale_2opt.sum()
        cigale_2nir += sources_cigale_2nir.sum()
        
        photoz += sources_photoz.sum() 
        cigale += sources_cigale.sum()
        both   += sources_both.sum() 
        
        photoz_just_r    += sources_photoz_just_r.sum() 
        photoz_just_4b   += sources_photoz_just_4b.sum() 
        cigale_just_2opt += sources_cigale_just_2opt.sum()
        cigale_just_2nir += sources_cigale_just_2nir.sum()

        n += 1
    
        total_obj += len(sub_catalogue)

    percentage_photoz_r    = photoz_r / total_obj
    percentage_photoz_4b   = photoz_4b / total_obj
    percentage_cigale_2opt = cigale_2opt / total_obj 
    percentage_cigale_2nir = cigale_2nir / total_obj 
    
    percentage_photoz = photoz / total_obj
    percentage_cigale = cigale / total_obj   
    percentage_both   = both / total_obj
    
    percentage_photoz_just_r    = photoz_just_r / total_obj
    percentage_photoz_just_4b   = photoz_just_4b / total_obj
    percentage_cigale_just_2opt = cigale_just_2opt / total_obj
    percentage_cigale_just_2nir = cigale_just_2nir / total_obj

#     percentage_photoz_r    = photoz_r / len(masterlist)
#     percentage_photoz_4b   = photoz_4b / len(masterlist)
#     percentage_cigale_2opt = cigale_2opt / len(masterlist) 
#     percentage_cigale_2nir = cigale_2nir / len(masterlist) 
    
#     percentage_photoz = photoz / len(masterlist)
#     percentage_cigale = cigale / len(masterlist)    
#     percentage_both   = both / len(masterlist)
    
#     percentage_photoz_just_r    = photoz_just_r / len(masterlist)
#     percentage_photoz_just_4b   = photoz_just_4b / len(masterlist)
#     percentage_cigale_just_2opt = cigale_just_2opt / len(masterlist) 
#     percentage_cigale_just_2nir = cigale_just_2nir / len(masterlist) 

    print("field: ", field)
    print("total objects in masterlist: ", len(masterlist))
    print("percentage sources photoz_4band sel: {:.3%}".format(percentage_photoz_r))
    print("percentage sources photoz_r sel: {:.3%}".format(percentage_photoz_4b))
    print("percentage sources cigale_2opt sel: {:.3%}".format(percentage_cigale_2opt))
    print("percentage sources cigale_2nir sel: {:.3%}".format(percentage_cigale_2nir))
    print("||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||")
    print("percentage sources photoz sel: {:.3%}".format(percentage_photoz))
    print("percentage sources cigale sel: {:.3%}".format(percentage_cigale))
    print("percentage sources both sel: {:.3%}".format(percentage_both))
    print("||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||")
    print("percentage sources with just photoz_4bands (not r detection): {:.3%}".format(percentage_photoz_just_r))
    print("percentage sources with just photoz_r (not 4bands detection): {:.3%}".format(percentage_photoz_just_4b))
    print("percentage sources with just cigale_2opt (not 2nir detection): {:.3%}".format(percentage_cigale_just_2opt))
    print("percentage sources with just cigale_2nir (not 2opt detection): {:.3%}".format(percentage_cigale_just_2nir))
    
    return len(masterlist), photoz, cigale, both

In [14]:
# Read original masterlist file

masterfile = '../../dmu1/dmu1_ml_ELAIS-N1/data/master_catalogue_elais-n1_20171016.fits'
masterlist = Table.read(masterfile, memmap=True)

bands = []
for col in masterlist.colnames:
    if col.startswith('f_') and not col.startswith('f_ap'):
        bands.append(col)
        
print('bands: ' , bands)

bands:  ['f_wfc_u', 'f_wfc_g', 'f_wfc_r', 'f_wfc_i', 'f_wfc_z', 'f_megacam_u', 'f_megacam_g', 'f_megacam_r', 'f_megacam_z', 'f_suprime_g', 'f_suprime_r', 'f_suprime_i', 'f_suprime_z', 'f_suprime_y', 'f_suprime_n921', 'f_gpc1_g', 'f_gpc1_r', 'f_gpc1_i', 'f_gpc1_z', 'f_gpc1_y', 'f_ukidss_j', 'f_ukidss_k', 'f_irac_i3', 'f_irac_i4', 'f_irac_i1', 'f_irac_i2']


In [15]:
all_bands = [['f_wfc_u',
 'f_megacam_u'],
 ['f_wfc_g',
 'f_megacam_g',
 'f_suprime_g',
 'f_gpc1_g'],
 ['f_wfc_r',
 'f_megacam_r',
 'f_suprime_r',
 'f_gpc1_r'], 
#['f_suprime_n921'],
 ['f_wfc_i',
 'f_suprime_i'],
 #'f_gpc1_i'],
 ['f_wfc_z',
 'f_megacam_z',
 'f_suprime_z'],
 #'f_gpc1_z'],
 ['f_suprime_y',
 'f_gpc1_y'],
 ['f_ukidss_j'],
 ['f_ukidss_k'],
 ['f_irac_i3'],
 ['f_irac_i4'],
 ['f_irac_i1'],
 ['f_irac_i2']]

## ELAIS-N1
## CIGALE bands used
#g_bands = ['suprime_g', 'megacam_g', 'wfc_g', 'gpc1_g']
#u_bands = ['megacam_u', 'wfc_u']
#r_bands = ['suprime_r', 'megacam_r', 'wfc_r', 'gpc1_r']
#i_bands = ['suprime_i', 'wfc_i']
#z_bands = ['suprime_z', 'megacam_z', 'wfc_z']
#y_bands = ['suprime_y', 'gpc1_y']


## photo-z bands used
# megacam, ukidss, irac


In [16]:
total_obj_en1, photoz_en1, cigale_en1, both_en1 = percentages_help_pipeline(masterlist, all_bands, 'ELAIS-N1')

num files:  41
field:  ELAIS-N1
total objects in masterlist:  4026292
percentage sources photoz_4band sel: 79.562%
percentage sources photoz_r sel: 71.777%
percentage sources cigale_2opt sel: 71.241%
percentage sources cigale_2nir sel: 12.631%
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
percentage sources photoz sel: 70.995%
percentage sources cigale sel: 11.504%
percentage sources both sel: 11.421%
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
percentage sources with just photoz_4bands (not r detection): 8.572%
percentage sources with just photoz_r (not 4bands detection): 0.787%
percentage sources with just cigale_2opt (not 2nir detection): 59.737%
percentage sources with just cigale_2nir (not 2opt detection): 1.127%


In [11]:
total_obj_en1, photoz_en1, cigale_en1, both_en1 = percentages_help_pipeline(masterlist, all_bands, 'ELAIS-N1')

num files:  41
field:  ELAIS-N1
total objects in masterlist:  4026292
percentage sources photoz_4band sel: 75.007%
percentage sources photoz_r sel: 71.874%
percentage sources cigale_2opt sel: 71.241%
percentage sources cigale_2nir sel: 12.631%
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
percentage sources photoz sel: 70.741%
percentage sources cigale sel: 11.504%
percentage sources both sel: 11.313%
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
percentage sources with just photoz_4bands (not r detection): 4.286%
percentage sources with just photoz_r (not 4bands detection): 1.154%
percentage sources with just cigale_2opt (not 2nir detection): 59.737%
percentage sources with just cigale_2nir (not 2opt detection): 1.127%


In [10]:
# Check sources in cigale catalogues
cigale_cat_en1 = Table.read('../../dmu28/dmu28_ELAIS-N1/data/ELAIS_N1_Ldust_prediction_results.fits', memmap=True)

len(cigale_cat_en1) / len(masterlist) * 100

11.358440967520487

In [6]:
# Check sources in cigale catalogues
photoz_cat_en1 = Table.read('../../dmu24/dmu24_ELAIS-N1/data/master_catalogue_elais-n1_20170706_photoz_20170725_irac1_optimised.fits', memmap=True)

len(photoz_cat_en1) / len(masterlist) * 100

71.65689920154821

In [14]:
cat_en1_photoz_flag = add_photoz_selection_flag(masterlist, all_bands)

In [15]:
photo_z_flag = ((cat_en1_photoz_flag['flag_photoz_sel_det'] == 3) | (~np.isnan(cat_en1_photoz_flag['zspec'])))

In [16]:
photo_z_flag.sum() / len(masterlist) * 100

70.74057221880578